In [4]:
import numpy as np
import cv2
import os

In [3]:
abs_dir_path = "/home/emanuele/ComputerVision/CV-CameraCalibration/videos/"
camera_number = 1
frame_count = 0
frame_skip = 30
# Open the video file
video_capture = cv2.VideoCapture(f'{abs_dir_path}out{camera_number}.mp4')

# Print the number of frames in the video
numberOf_frame = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
print("Number of frames in the video: ", numberOf_frame)

Number of frames in the video:  13628


In [ ]:
# Create a directory to save the screen captures named as the video file
output_dir = 'samples/' + VIDEO_NAME
os.makedirs(output_dir, exist_ok=True)
print("Saving frames to ", output_dir)
# Remove all files in the directory
for file in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, file))



In [ ]:


while True:
        # Read a frame from the video
        ret, img = video_capture.read()
        if not ret:
            break  # Break the loop if we've reached the end of the video

        # Skip frames based on the frame_skip value
        if frame_count % frame_skip == 0:
            # Resize the frame to the new_resolution
            real_img = img
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Find the chessboard corners
            found_corners, corners = cv2.findChessboardCorners(gray, (CHESS_WIDTH, CHESS_HEIGHT), None)
            #print("Frame ", frame_count, ":", found_corners)